In [3]:
import pandas as pd
import requests
import json
import time

# === CONFIG ===
API_KEY = "cqt_rQpBCvkvR4rBckGBKkxYjYQt7Pgb"  # Replace with your real key
CHAIN_ID = "1"  # 1 = Ethereum mainnet
CSV_FILE = "Wallet id - Sheet1.csv"

# === Load wallet addresses ===
wallet_df = pd.read_csv(CSV_FILE)
wallet_addresses = wallet_df.iloc[:, 0].dropna().unique().tolist()

# === Covalent API Call ===
def fetch_wallet_transactions(wallet_address):
    url = f"https://api.covalenthq.com/v1/{CHAIN_ID}/address/{wallet_address}/transactions_v2/?key={API_KEY}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Error fetching {wallet_address}: {e}")
        return None

# === Collect data ===
wallet_transactions = {}

for i, wallet in enumerate(wallet_addresses):
    print(f"🔄 Fetching transactions for {wallet} ({i+1}/{len(wallet_addresses)})")
    result = fetch_wallet_transactions(wallet)
    if result and result.get("data", {}).get("items"):
        wallet_transactions[wallet] = result["data"]["items"]
    time.sleep(1)  # prevent throttling

# === Save to JSON ===
with open("wallet_transactions.json", "w") as f:
    json.dump(wallet_transactions, f, indent=2)

print("✅ Saved all transaction histories to wallet_transactions.json")


🔄 Fetching transactions for 0x0039f22efb07a647557c7c5d17854cfd6d489ef3 (1/103)
🔄 Fetching transactions for 0x06b51c6882b27cb05e712185531c1f74996dd988 (2/103)
🔄 Fetching transactions for 0x0795732aacc448030ef374374eaae57d2965c16c (3/103)
🔄 Fetching transactions for 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9 (4/103)
🔄 Fetching transactions for 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae (5/103)
🔄 Fetching transactions for 0x104ae61d8d487ad689969a17807ddc338b445416 (6/103)
🔄 Fetching transactions for 0x111c7208a7e2af345d36b6d4aace8740d61a3078 (7/103)
🔄 Fetching transactions for 0x124853fecb522c57d9bd5c21231058696ca6d596 (8/103)
🔄 Fetching transactions for 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc (9/103)
🔄 Fetching transactions for 0x1656f1886c5ab634ac19568cd571bc72f385fdf7 (10/103)
🔄 Fetching transactions for 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22 (11/103)
🔄 Fetching transactions for 0x19df3e87f73c4aaf4809295561465b993e102668 (12/103)
🔄 Fetching transactions for 0x1ab2ccad4fc97c9968e

In [4]:
import json

# Load the file
with open("wallet_transactions.json", "r") as f:
    data = json.load(f)

transaction_types = set()

for wallet, transactions in data.items():
    for tx in transactions:
        for log in tx.get("log_events", []):
            decoded = log.get("decoded")
            if decoded and "name" in decoded:
                transaction_types.add(decoded["name"])

# Print unique transaction types
cnt=0
print("Unique transaction types:")
t=[]
for _ in sorted(transaction_types):
    t.append(_)
    cnt+=1
print(t)
print(cnt)

Unique transaction types:
['AbsorbCollateral', 'AbsorbDebt', 'AccrueInterest', 'ActionEvent', 'AddLiquidity', 'AddrChanged', 'AddressChanged', 'AnchorPriceUpdated', 'Approval', 'ApprovalForAll', 'AssetIndexUpdated', 'AssignBurnFees', 'AssignFeeToWallet', 'AuctionBid', 'AuctionCancelled', 'AuctionCreated', 'AuctionSuccessful', 'BalanceTransfer', 'Borrow', 'BorrowAllowanceDelegated', 'BreedingPotionCheckpoint', 'Burn', 'Burned', 'Buy', 'BuyCollateral', 'BuyGem', 'ClaimRewards', 'Claimed', 'ClaimedTokens', 'Collect', 'Conversion', 'Cooldown', 'Created', 'DecreaseLiquidity', 'DelegateChanged', 'DelegateVotesChanged', 'DelegatedPowerChanged', 'Deposit', 'DepositInitiated', 'DepositReceived', 'Deposited', 'DistributedBorrowerComp', 'DistributedSupplierComp', 'DistributionClaimed', 'Donate', 'ETHSubmitted', 'EnabledStaticCall', 'EntryFeePaid', 'Error', 'EthPurchase', 'EtherDeposited', 'EtherReceival', 'ExecuteTrade', 'Executed', 'ExecutionSuccess', 'ExternalBalanceTransfer', 'Failure', 'FeesC

In [ ]:
import json
import pandas as pd
import numpy as np
from datetime import datetime
import logging
from typing import Dict

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def wei_to_ether(wei_amount, decimals=18):
    """Convert wei to ether-like units based on token decimals."""
    try:
        return float(wei_amount) / (10 ** decimals)
    except (ValueError, TypeError):
        logging.warning(f"Invalid wei amount: {wei_amount}")
        return 0.0

def normalize_feature(series: pd.Series, method: str = 'minmax') -> pd.Series:
    """Normalize a series to [0, 1] using min-max or logarithmic scaling."""
    if method == 'log':
        series = np.log1p(series)
        if series.max() == series.min():
            return pd.Series(0.0, index=series.index)
        return (series - series.min()) / (series.max() - series.min())
    else:
        if series.max() == series.min():
            return pd.Series(0.0, index=series.index)
        return (series - series.min()) / (series.max() - series.min())

def get_amount_from_params(params: list, event: str) -> float:
    """Extract amount from event parameters, handling various parameter names."""
    param_dict = {p["name"]: p["value"] for p in params if isinstance(p, dict) and "name" in p and "value" in p}
    amount_keys = ['amount', 'value', 'wad', 'amount0', 'amount1', 'collateralAmount', 
                   'supplyAmount', 'withdrawAmount', 'repayAmount', 'borrowAmount', 'debtAmount']
    for key in amount_keys:
        if key in param_dict:
            try:
                return float(param_dict[key])
            except (ValueError, TypeError):
                logging.warning(f"Invalid amount for key {key} in event {event}")
                return 0.0
    logging.warning(f"No amount found for event {event}")
    return 0.0

def calculate_credit_score(stats_df: pd.DataFrame) -> pd.Series:
    """
    Calculate credit score (0-1000) based on weighted normalized features.
    Higher score indicates safer credit profile.
    """
    weights = {
        'total_deposit_usd': 0.12,        # High deposits reduce risk
        'total_repay_usd': 0.10,         # Consistent repayments reduce risk
        'num_deposits': 0.07,            # Frequent deposits indicate engagement
        'num_repays': 0.07,              # Frequent repayments indicate reliability
        'total_comp_received': 0.07,     # COMP rewards suggest long-term engagement
        'unique_tx_types': 0.05,         # Diverse interactions reduce risk
        'compound_core_tx_count': 0.08,  # Active Compound participation reduces risk
        'compound_reward_tx_count': 0.07,  # Reward events indicate positive engagement
        'interest_event_count': 0.05,    # Interest accrual shows market participation
        'collateral_management_count': 0.08,  # Collateral management reduces risk
        'defi_interaction_count': 0.05,  # Broader DeFi engagement reduces risk
        'staking_voting_count': 0.05,    # Governance/staking shows engagement
        'defi_diversity_score': 0.05,    # Diverse DeFi interactions reduce risk
        'compound_core_tx_ratio': 0.05,  # Focus on Compound protocol reduces risk
        'collateral_to_borrow_ratio': 0.06,  # Strong collateralization reduces risk
        'debt_to_collateral_ratio': -0.10,  # High leverage increases risk
        'num_liquidations': -0.12,         # Liquidations strongly increase risk
        'liquidation_to_tx_ratio': -0.08,  # High liquidation ratio increases risk
        'avg_borrow_per_tx': -0.07,       # Large borrowings increase risk
        'negative_event_count': -0.10     # Errors or risky events increase risk
    }

    # Normalize features
    normalized_df = pd.DataFrame()
    for col, weight in weights.items():
        if col in ['total_deposit_usd', 'total_repay_usd', 'total_comp_received', 'avg_borrow_per_tx']:
            normalized_df[col] = normalize_feature(stats_df[col], method='log')
        else:
            normalized_df[col] = normalize_feature(stats_df[col], method='minmax')

    # Invert risky features
    for col in ['debt_to_collateral_ratio', 'num_liquidations', 'liquidation_to_tx_ratio', 
                'avg_borrow_per_tx', 'negative_event_count']:
        normalized_df[col] = 1 - normalized_df[col]

    # Calculate weighted sum
    score = sum(normalized_df[col] * weight for col, weight in weights.items())
    
    # Scale to 0-1000
    score = score.clip(lower=0, upper=1) * 1000
    return score

try:
    # Load JSON data
    with open("compound_transactions.json", "r") as f:
        data = json.load(f)
except FileNotFoundError:
    logging.error("compound_transactions.json file not found")
    exit(1)
except json.JSONDecodeError:
    logging.error("Invalid JSON format in compound_transactions.json")
    exit(1)
except MemoryError:
    logging.error("Memory error loading 800MB JSON. Consider using ijson for incremental parsing.")
    exit(1)

# Define transaction type categories
COMPOUND_CORE_EVENTS = {
    'Mint', 'Borrow', 'RepayBorrow', 'Redeem', 'LiquidateBorrow', 'MarketEntered', 'MarketExited',
    'LogBorrow', 'LogRepay', 'LogPoolBorrow', 'LogPoolPayback', 'AbsorbDebt'
}
COMPOUND_REWARD_EVENTS = {
    'DistributedSupplierComp', 'DistributedBorrowerComp', 'ClaimRewards', 'RewardsClaimed', 'DistributionClaimed'
}
INTEREST_EVENTS = {'AccrueInterest'}
COLLATERAL_EVENTS = {
    'Mint', 'Deposit', 'Supply', 'AddLiquidity', 'SupplyCollateral', 'DepositReceived', 
    'Deposited', 'DepositInitiated', 'LogDeposit', 'LogAddCollateral',
    'Redeem', 'Withdraw', 'WithdrawCollateral', 'RemoveLiquidityImbalance', 
    'RemoveLiquidityOne', 'DecreaseLiquidity', 'LogRemoveCollateral', 'LogWithdraw', 
    'Withdrawn', 'Withdrawal', 'AbsorbCollateral'
}
DEFI_EVENTS = {
    'Swap', 'FlashLoan', 'UniswapWindowUpdated', 'TokenExchange', 'TokenExchangeUnderlying',
    'KyberTrade', 'OrderFilled', 'OrdersMatched', 'Fill', 'SellBaseToken', 'Buy', 
    'TokenPurchase', 'Trade', 'TradeExecute', 'Swapped', 'Sync', 'TokensTraded',
    'LOG_SWAP', 'Swapp', 'SwapSTRP', 'FillOrder', 'AuctionBid', 'AuctionCreated',
    'AuctionSuccessful', 'AuctionCancelled'
}
TOKEN_EVENTS = {
    'Transfer', 'Approval', 'TokensTransferred', 'TransferSingle', 'TransferShares',
    'TransferByPartition', 'BalanceTransfer', 'ExternalBalanceTransfer', 'LockedERC20',
    'TokensMinted', 'TokensBurned', 'Burn', 'Burned', 'MintFrozen', 'TransferredToken'
}
NEGATIVE_EVENTS = {'Failure', 'Error', 'LiquidateBorrow', 'AbsorbCollateral', 'AbsorbDebt'}
STAKING_VOTING_EVENTS = {
    'Stake', 'Unstake', 'VoteCast', 'Voted', 'DelegateChanged', 'DelegateVotesChanged',
    'Staked', 'MoveStake', 'DelegatedPowerChanged'
}

wallet_stats = {}

for wallet, txns in data.items():
    stats = {
        "total_deposit_usd": 0.0,
        "total_borrow_usd": 0.0,
        "total_repay_usd": 0.0,
        "total_redeem_usd": 0.0,
        "num_liquidations": 0,
        "num_deposits": 0,
        "num_borrows": 0,
        "num_repays": 0,
        "num_redeems": 0,
        "num_transfers": 0,
        "unique_tx_types": set(),
        "total_tx_count": 0,
        "total_comp_received": 0.0,
        "avg_borrow_per_tx": 0.0,
        "avg_repay_per_tx": 0.0,
        "compound_core_tx_count": 0,
        "compound_reward_tx_count": 0,
        "interest_event_count": 0,
        "collateral_management_count": 0,
        "defi_interaction_count": 0,
        "token_interaction_count": 0,
        "negative_event_count": 0,
        "staking_voting_count": 0,
        "defi_diversity_score": 0,
        "compound_core_tx_ratio": 0.0,
        "collateral_to_borrow_ratio": 0.0,
        "last_tx_timestamp": None
    }
    defi_tx_types = set()

    for tx in txns:
        stats["total_tx_count"] += 1
        log_events = tx.get("log_events", [])
        
        # Update last transaction timestamp (for reference)
        tx_time = tx.get("block_signed_at")
        if tx_time:
            try:
                tx_timestamp = datetime.fromisoformat(tx_time.replace("Z", "+00:00"))
                if stats["last_tx_timestamp"] is None or tx_timestamp > stats["last_tx_timestamp"]:
                    stats["last_tx_timestamp"] = tx_timestamp
            except ValueError:
                logging.warning(f"Invalid timestamp format: {tx_time}")

        for log in log_events:
            decoded = log.get("decoded")
            if not decoded or not isinstance(decoded, dict):
                continue

            tx_type = decoded.get("name")
            if not tx_type:
                continue
            stats["unique_tx_types"].add(tx_type)

            params = decoded.get("params", [])
            if not isinstance(params, list):
                continue

            # Get token decimals from log
            decimals = log.get("sender_contract_decimals", 18) or 18
            if log.get("sender_address", "").lower() == "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48":  # USDC
                decimals = 6

            # Handle transaction types
            if tx_type in {'Mint', 'Deposit', 'Supply', 'AddLiquidity', 'SupplyCollateral', 
                           'DepositReceived', 'Deposited', 'DepositInitiated', 'LogDeposit', 
                           'LogAddCollateral'}:
                stats["total_deposit_usd"] += wei_to_ether(get_amount_from_params(params, tx_type), decimals=decimals)
                stats["num_deposits"] += 1
                stats["collateral_management_count"] += 1
                if tx_type in COMPOUND_CORE_EVENTS:
                    stats["compound_core_tx_count"] += 1
            elif tx_type in {'Redeem', 'Withdraw', 'WithdrawCollateral', 'RemoveLiquidityImbalance', 
                             'RemoveLiquidityOne', 'DecreaseLiquidity', 'LogRemoveCollateral', 
                             'LogWithdraw', 'Withdrawn', 'Withdrawal'}:
                stats["total_redeem_usd"] += wei_to_ether(get_amount_from_params(params, tx_type), decimals=decimals)
                stats["num_redeems"] += 1
                stats["collateral_management_count"] += 1
                if tx_type in COMPOUND_CORE_EVENTS:
                    stats["compound_core_tx_count"] += 1
            elif tx_type in {'Borrow', 'LogBorrow', 'LogPoolBorrow', 'AbsorbDebt'}:
                stats["total_borrow_usd"] += wei_to_ether(get_amount_from_params(params, tx_type), decimals=decimals)
                stats["num_borrows"] += 1
                stats["compound_core_tx_count"] += 1
                if tx_type == 'AbsorbDebt':
                    stats["negative_event_count"] += 1
            elif tx_type in {'RepayBorrow', 'Repay', 'LogRepay', 'LogPoolPayback'}:
                stats["total_repay_usd"] += wei_to_ether(get_amount_from_params(params, tx_type), decimals=decimals)
                stats["num_repays"] += 1
                stats["compound_core_tx_count"] += 1
            elif tx_type == "LiquidateBorrow":
                stats["num_liquidations"] += 1
                stats["compound_core_tx_count"] += 1
                stats["negative_event_count"] += 1
            elif tx_type == "AbsorbCollateral":
                stats["num_liquidations"] += 1
                stats["collateral_management_count"] += 1
                stats["negative_event_count"] += 1
            elif tx_type == "Transfer":
                stats["num_transfers"] += 1
                stats["token_interaction_count"] += 1
            elif tx_type in COMPOUND_REWARD_EVENTS:
                stats["total_comp_received"] += wei_to_ether(get_amount_from_params(params, tx_type), decimals=18)
                stats["compound_reward_tx_count"] += 1
            elif tx_type in INTEREST_EVENTS:
                stats["interest_event_count"] += 1
            elif tx_type in COLLATERAL_EVENTS:
                stats["collateral_management_count"] += 1
            elif tx_type in DEFI_EVENTS:
                stats["defi_interaction_count"] += 1
                defi_tx_types.add(tx_type)
            elif tx_type in TOKEN_EVENTS:
                stats["token_interaction_count"] += 1
            elif tx_type in NEGATIVE_EVENTS:
                stats["negative_event_count"] += 1
            elif tx_type in STAKING_VOTING_EVENTS:
                stats["staking_voting_count"] += 1
            elif tx_type in COMPOUND_CORE_EVENTS:
                stats["compound_core_tx_count"] += 1

    stats["avg_borrow_per_tx"] = (
        stats["total_borrow_usd"] / stats["num_borrows"]
        if stats["num_borrows"] > 0
        else 0.0
    )
    stats["avg_repay_per_tx"] = (
        stats["total_repay_usd"] / stats["num_repays"]
        if stats["num_repays"] > 0
        else 0.0
    )
    stats["debt_to_collateral_ratio"] = (
        stats["total_borrow_usd"] / stats["total_deposit_usd"]
        if stats["total_deposit_usd"] > 0
        else stats["total_borrow_usd"]
    )
    stats["liquidation_to_tx_ratio"] = (
        stats["num_liquidations"] / stats["total_tx_count"]
        if stats["total_tx_count"] > 0
        else 0.0
    )
    stats["compound_core_tx_ratio"] = (
        stats["compound_core_tx_count"] / stats["total_tx_count"]
        if stats["total_tx_count"] > 0
        else 0.0
    )
    stats["collateral_to_borrow_ratio"] = (
        stats["collateral_management_count"] / stats["num_borrows"]
        if stats["num_borrows"] > 0
        else stats["collateral_management_count"]
    )
    stats["defi_diversity_score"] = len(defi_tx_types)
    stats["unique_tx_types"] = len(stats["unique_tx_types"])

    wallet_stats[wallet] = stats

wallet_df = pd.DataFrame.from_dict(wallet_stats, orient="index").reset_index()
wallet_df = wallet_df.rename(columns={"index": "wallet_address"})

wallet_df['credit_score'] = calculate_credit_score(wallet_df)

try:
    wallet_df.to_csv("compound_wallet_stats.csv", index=False)
    logging.info("Results with credit scores exported to compound_wallet_stats.csv")
except Exception as e:
    logging.error(f"Failed to export to CSV: {str(e)}")

2025-07-28 04:32:56,117 - WARNING - No amount found for event DistributedSupplierComp
2025-07-28 04:32:56,118 - WARNING - No amount found for event DistributedSupplierComp
2025-07-28 04:32:56,118 - WARNING - No amount found for event DistributedSupplierComp
2025-07-28 04:32:56,118 - WARNING - No amount found for event DistributedSupplierComp
2025-07-28 04:32:56,119 - WARNING - No amount found for event DistributedSupplierComp
2025-07-28 04:32:56,119 - WARNING - No amount found for event DistributedSupplierComp
2025-07-28 04:32:56,119 - WARNING - No amount found for event DistributedSupplierComp
2025-07-28 04:32:56,120 - WARNING - No amount found for event DistributedSupplierComp
2025-07-28 04:32:56,120 - WARNING - No amount found for event Redeem
2025-07-28 04:32:56,120 - WARNING - No amount found for event DistributedSupplierComp
2025-07-28 04:32:56,120 - WARNING - No amount found for event DistributedSupplierComp
2025-07-28 04:32:56,121 - WARNING - No amount found for event Distribut